# Prácticas ESAC: Strömgren Radius Jung20

1. Extracted data from Jung20
2. Filter in redshift
3. Strömgen Radius

   Appendix A: Saving data

In [9]:
# Paquetes
from astropy.table import Table, QTable, unique
from astropy.io import ascii
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import smplotlib
from uncertainties import ufloat, unumpy
import uncertainties.unumpy as unp
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import sys

# Definimos Mpc en unidades propias comóviles
pMpc = u.def_unit('pMpc')
cMpc = u.def_unit('cMpc')

## 1. Extracted data from Jung20

A continuación, importamos la _Table 1_ del paper 11 (Intae Jung et al. (2020)). Es posible que si busca esta tabla en internet no la encuentre puesto que aparece sólo en la versión 2 (v2) del paper (última versión a fecha en la que se escribe esto). 

In [2]:
# Creamos la tabla de datos de Jung20
data_Jung20 = {
    'ID': ['z7_GND_44088', 'z8_GND_22233', 'z7_GND_18626', 'z7_GND_42912', 'z7_GND_6330', 'z7_GND_16863', 'z7_GND_34204', 'z7_GND_7376', 'z7_GND_39781', 'z7_GND_10402','z7_GND_6451', 'z7_GND_45190', 'z8_GND_41470', 'z8_GND_41247', 'z7_GND_7157'],
    'logFLya': np.round(np.log10([1.27*10**(-17), 1.36*10**(-17), 0.26*10**(-17), 1.46*10**(-17), 0.41*10**(-17), 1.89*10**(-17), 4.51*10**(-17), 0.26*10**(-17), 1.60*10**(-17), 0.32*10**(-17), 0.68*10**(-17), 0.31*10**(-17), 0.93*10**(-17), 1.70*10**(-17), 0.26*10**(-17)]),3)*u.dex(u.erg/u.s),
    'e_logFLya': np.round([0.25/1.27, 0.19/1.36,0.06/0.26, 0.13/1.46, 0.07/0.41, 0.18/1.89, 0.57/4.51, 0.06/0.26, 0.35/1.60, 0.08/0.32,0.21/0.68, 0.09/0.31, 0.26/0.93, 0.44/1.70, 0.08/0.26],3)*u.dex(u.erg/u.s),
    'S/N': [5.2, 7.1, 4.6, 10.8, 6.1, 10.8, 7.9, 4.1, 4.5, 4.0,3.2, 3.4, 3.5, 3.9, 3.4],
    'EW_Lya': [87.6, 54.5, 26.8, 33.2, 15.9, 61.3, 279.7, 32.5, 123.9, 6.7,43.2, 22.9, 25.9, 164.2, 21.2]*u.angstrom,
    'E_EW_Lya': [23.8, 15.0, 14.9, 4.3, 4.4, 14.4, 80.4, 23.0, 37.4, 2.7,17.2, 13.7, 9.5, 85.8, 9.9]*u.angstrom,
    'e_EW_Lya': [21.2, 12.1, 9.8, 4.0, 3.7, 11.4, 62.5, 13.0, 32.9, 2.2,15.1, 9.3, 8.4, 60.5, 8.0]*u.angstrom,
    'z_Lya': [7.1335, 7.3444, 7.4249, 7.5056, 7.5460, 7.5989, 7.6082, 7.7681, 7.8809, 7.9395,7.2462, 7.2650, 7.3115, 8.0356, 8.1280],
    'e_z_Lya': [0.0028, 0.0020, 0.0013, 0.0007, 0.0006, 0.0011, 0.0030, 0.0024, 0.0018, 0.0023,0.0045, 0.0019, 0.0028, 0.0015, 0.0016],
    'logLLya': np.round(np.log10([0.79*10**(43), 0.91*10**(43), 0.18*10**(43), 1.02*10**(43), 0.29*10**(43), 1.36*10**(43), 3.26*10**(43), 0.20*10**(43), 1.25*10**(43), 0.26*10**(43),0.44*10**(43), 0.20*10**(43), 0.61*10**(43), 1.39*10**(43), 0.22*10**(43)]),3)*u.dex(u.erg/u.s),
    'e_logLLya': np.round([0.16/0.79, 0.13/0.81, 0.04/0.18, 0.09/1.02, 0.05/0.29, 0.13/1.36, 0.41/3.26, 0.05/0.20, 0.27/1.25, 0.06/0.26,0.14/0.44, 0.06/0.20, 0.17/0.61, 0.36/1.39, 0.07/0.22],3)*u.dex(u.erg/u.s),
    'HII_Radii': [0.88, 0.92, 0.52, 0.96, 0.62, 1.07, 1.44, 0.54, 1.04, 0.59,0.72, 0.55, 0.81, 1.07, 0.56]*pMpc,
    'M_UV': [-19.9, -20.7, -19.8, -21.6, -20.7, -21.2, -20.4, -19.5, -20.2, -21.7,-20.0, -20.4, -21.2, -20.2, -20.5]*u.mag,
}

Jung20 = Table(data_Jung20)
Jung20

ID,logFLya,e_logFLya,S/N,EW_Lya,E_EW_Lya,e_EW_Lya,z_Lya,e_z_Lya,logLLya,e_logLLya,HII_Radii,M_UV
,dex(erg / s),dex(erg / s),,Angstrom,Angstrom,Angstrom,,,dex(erg / s),dex(erg / s),pMpc,mag
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z7_GND_44088,-16.896,0.197,5.2,87.6,23.8,21.2,7.1335,0.0028,42.898,0.203,0.88,-19.9
z8_GND_22233,-16.866,0.14,7.1,54.5,15.0,12.1,7.3444,0.002,42.959,0.16,0.92,-20.7
z7_GND_18626,-17.585,0.231,4.6,26.8,14.9,9.8,7.4249,0.0013,42.255,0.222,0.52,-19.8
z7_GND_42912,-16.836,0.089,10.8,33.2,4.3,4.0,7.5056,0.0007,43.009,0.088,0.96,-21.6
z7_GND_6330,-17.387,0.171,6.1,15.9,4.4,3.7,7.546,0.0006,42.462,0.172,0.62,-20.7
z7_GND_16863,-16.724,0.095,10.8,61.3,14.4,11.4,7.5989,0.0011,43.134,0.096,1.07,-21.2
...,...,...,...,...,...,...,...,...,...,...,...,...
z7_GND_10402,-17.495,0.25,4.0,6.7,2.7,2.2,7.9395,0.0023,42.415,0.231,0.59,-21.7


## 2. Filter in redshift

Utilzaremos los datos de Hu18 para la realización de este cuaderno escogiendo únicamente aquellas galaxias con z > 6, puesto que estas son las únicas para las cuales el Universo todavía no estaba completamente reionizado y por tanto, estas esferas tenían un tamaño finito (no ocupaban todo el universo). 

In [3]:
Jung20_filter = Jung20[Jung20['z_Lya'] > 6]
Jung20_filter

ID,logFLya,e_logFLya,S/N,EW_Lya,E_EW_Lya,e_EW_Lya,z_Lya,e_z_Lya,logLLya,e_logLLya,HII_Radii,M_UV
,dex(erg / s),dex(erg / s),,Angstrom,Angstrom,Angstrom,,,dex(erg / s),dex(erg / s),pMpc,mag
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z7_GND_44088,-16.896,0.197,5.2,87.6,23.8,21.2,7.1335,0.0028,42.898,0.203,0.88,-19.9
z8_GND_22233,-16.866,0.14,7.1,54.5,15.0,12.1,7.3444,0.002,42.959,0.16,0.92,-20.7
z7_GND_18626,-17.585,0.231,4.6,26.8,14.9,9.8,7.4249,0.0013,42.255,0.222,0.52,-19.8
z7_GND_42912,-16.836,0.089,10.8,33.2,4.3,4.0,7.5056,0.0007,43.009,0.088,0.96,-21.6
z7_GND_6330,-17.387,0.171,6.1,15.9,4.4,3.7,7.546,0.0006,42.462,0.172,0.62,-20.7
z7_GND_16863,-16.724,0.095,10.8,61.3,14.4,11.4,7.5989,0.0011,43.134,0.096,1.07,-21.2
...,...,...,...,...,...,...,...,...,...,...,...,...
z7_GND_10402,-17.495,0.25,4.0,6.7,2.7,2.2,7.9395,0.0023,42.415,0.231,0.59,-21.7


## 3. Strömgen Radius

Ya es el momento de estimar el radio de Strömgen, o mejor dicho, el límite superior al tamaño de estas burbujas para cada galaxia a z > 6. Este radio se define como:

$$
\begin{equation}
    R_s = \left(\frac{3 \dot{N}_{ion}}{4\pi C_H \langle n_H \rangle ^ 2 \alpha_B}\right)^{1/3}
\end{equation}
$$

donde $\dot{N}_{ion}$ se corresponde con el número de fotones del continuo ionizante, según las definciones en [Reionized-Bubble-published](https://watermark.silverchair.com/mnrasl_495_1_l17.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAA3UwggNxBgkqhkiG9w0BBwagggNiMIIDXgIBADCCA1cGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMesIRyAw1IMhjJgxLAgEQgIIDKNLnPEZWRv4H07eC47gxYs3YVUGv5PmSrfVEViCuE9OGjq7ro9MHhMyPmAakzFX36UUoujJ_iE9CnliM-I7rMh4z7mOeffZzdY6hfOyq00dea8-Lq1pWKHV4W24VMD60GNXdkNjd2aavHQncseS1xETCLGjWV75goKbuh_yg9nXTHI6v6S2MA0fKTlzk56zTvu3069rRq-WpUdLuyv0U6NSQNNtqfWMR468NbLLTfIHve9wfcSxGB-iHQiIJD0nQVAkDzO-LT1nxBzsiES0EvVeGqMNtyfLWRrMX8NxIJaaGEljDw5QQtk3ogx7j5QCeYpUBYY7__mlZBc8-pdDzOQ5ALXsasW69Qkf1sSmFzIvTPaMEK9YUN8axtA9iJViI5U33clBDs8xa0nJi5GA1IsnB9Elr1WP06eTj6v-HQeNqO5w4vP9vKIYTsDRvFzuu6U-Ok0Kw0aH2ybnJAVYQ8qfFRo29CcuPwsu8UVwCpISBnRl-269GGOHYzPoEzJrp_dWBnvXTYh6TTl0178RJhmDySOlH5Ie4tMfe-dp4cdQnhl0z-x4MQRIG_SDb3Xo-crH844EffwRQ5EnsktfdoMwXXvndvakMYFzSCSYEnJU1ZhMCUPH3IeyMQS1IXNM05Qm8QGW1OlUrqIvLDMjlCZeWptQ6X7mX5lX9jph4M9eKAelJw-dZjqlhm6NHjGS7FjhfsczkoehMAi87W5bm6FPbQlTpzEdDoNfMcc3GPH1SEbdltmN8j6TrfTA5pk2JFqntSzJJC99DHJzDeSWGumc1zg4FrP7vkUC4yAGgoMsSac6IL_SpwXEKc9sKB9ezu5PVy4NrWL7sc-xSmau0sVn8g_wP25TePV25N4Z8GxAE1cV_PPHKOFJU21pomgFaZdxyPxENLoyx9v64JuoXhi48PWuV9V1JPAXatkS2q3VTz5hRGiUD9nxqy7DbU6jVmZwqErVDNamfyN_0H7VxMEOm5sa65jZyHaGtyknwQq6fktLmskgK_kmotgnM2rZWexXFBGzK5USVEwHe-iA7Ri0pKvudfX8MtFVjs8HZGOqBvO118nGdD3Q). Esto es,

$$
\begin{equation}
    \dot{N}_{ion} = \frac{Q_{ion}}{1-f_{esc,LyC}} \cdot f_{esc,LyC}
\end{equation}
$$

donde $Q_{ion}$ se corresponde con los valores de $10^{logN_{ion}}$ de la tabla de Kerutt22 $\textbf{(cuidado con la notación)}$. Se asume $f_{esc,LyC} = 0.2$, valor estimador por diversos autores. 

Para realizar este cálculo, vamos a crear una función que lo calcule directamente a partir de la anchura equivalente (en angstroms), el logaritmo en base 10 de la luminosidad observada de la línea de lymann alpha (en dex(erg/s) y el redsfhit (adimensional). 

In [4]:
# Strömgen radius calculation using equivalent width (EW), log10(observed luminosity of Lymann alpha) and redshift
# Units: EW in angstroms; logLLya in dex(erg/s); z (adimensional)
def R_s(EW, E_EW, e_EW, logLLya, e_logLLya, redshift):

    # recombination coefficinet for T_e = 10⁴K
    alphaB = 8.8 * 10**(-87)

    # C_H(z) parameter
    def C_H(redshift):
        z_values = [6 ,7 , 8]
        C_H_values = [2.37, 2.28, 2.19]
        C_H = np.interp(redshift, z_values,C_H_values) 
        return C_H

    # Hydrogen density
    def n_H (redshift):
        n_H = 5.878 * 10**66 * (1+redshift)**3
        return n_H

    # Escape fraction of Lymann alpha
    f_esc_Lya = []

    for i in range(len(EW)):
        if 0 <= EW[i] <= 200:
            f_esc_Lya.append(0.0048*EW[i])
        else:
            f_esc_Lya.append(1)

    e_f_esc_Lya = 0.0048 * e_EW
    E_f_esc_Lya = 0.0048 * E_EW

    # log10(intrisic luminosity of Lymann alpha)
    logLLya_int = np.log10(10**logLLya/f_esc_Lya)
    e_logLLya_int = np.sqrt( ((e_logLLya*np.log(10)*10**logLLya)/f_esc_Lya)**2 + (e_f_esc_Lya/(10**logLLya))** 2); e_logLLya_int = e_logLLya_int / (10**logLLya * np.log(10))
    E_logLLya_int = np.sqrt( ((e_logLLya*np.log(10)*10**logLLya)/f_esc_Lya)**2 + (E_f_esc_Lya/(10**logLLya))** 2); E_logLLya_int = E_logLLya_int / (10**logLLya * np.log(10))
    
    # Effective number of ionizinz continuum photones per second
    logQ_ion = np.log10(10**logLLya_int / (1.19*10**(-11)))
    e_logQ_ion = (1/np.log(10)) * (1/10**logQ_ion) * ((e_logLLya_int*np.log(10)*10**logLLya_int)/(1.19*10**(-11)))
    E_logQ_ion = (1/np.log(10)) * (1/10**logQ_ion) * ((E_logLLya_int*np.log(10)*10**logLLya_int)/(1.19*10**(-11)))

    # Nuber of ionizing continuum photones that participate in reionizing the IGM
    f_esc_LyC = 0.2 # assumed f_esc_LyC
    N_ion = (10**logQ_ion) / (1 - f_esc_LyC) * f_esc_LyC
    e_logN_ion = (1/np.log(10)) * (1/N_ion) * (e_logQ_ion*np.log(10)*10**logQ_ion) * f_esc_LyC / (1 - f_esc_LyC)
    E_logN_ion = (1/np.log(10)) * (1/N_ion) * (E_logQ_ion*np.log(10)*10**logQ_ion) * f_esc_LyC / (1 - f_esc_LyC)

    # Strömgen radius (proper)
    R_s = (3 * N_ion / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB))**(1/3)
    e_R_s = 1/3 * (3 * (e_logN_ion * np.log(10) * N_ion) / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB)) * (3 * (N_ion) / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB))**(-2/3)
    E_R_s = 1/3 * (3 * (E_logN_ion * np.log(10) * N_ion) / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB)) * (3 * (N_ion) / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB))**(-2/3)

    # Strömgen radius (comovil)
    R_s_com = (1 + redshift) * R_s
    e_R_s_com = (1 + redshift) * e_R_s
    E_R_s_com = (1 + redshift) * E_R_s
    
    return np.round(R_s,3) * pMpc, np.round(R_s_com,3) * cMpc, np.round(f_esc_Lya,3), np.round(logLLya_int,3) * u.dex(u.erg / u.s), np.round(logQ_ion,3)  * u.dex(1 / u.s), np.round(EW,2)  * u.angstrom, np.round(np.log10(N_ion),3) * u.dex(1 / u.s), np.round(e_f_esc_Lya,3), np.round(E_f_esc_Lya,3), np.round(e_logLLya_int,33) * u.dex(u.erg / u.s), np.round(E_logLLya_int,33) * u.dex(u.erg / u.s), np.round(e_logN_ion,3) * u.dex(1 / u.s), np.round(E_logN_ion,3) * u.dex(1 / u.s), np.round(e_R_s,3) * pMpc, np.round(E_R_s,3) * pMpc, np.round(e_R_s_com,3) * cMpc, np.round(E_R_s_com,3) * cMpc

In [5]:
Jung20_filter['R_s'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[0]
Jung20_filter['e_R_s'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[13]
Jung20_filter['E_R_s'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[14]
Jung20_filter['R_s_com'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[1]
Jung20_filter['e_R_s_com'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[15]
Jung20_filter['E_R_s_com'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[16]
Jung20_filter['f_esc_Lya'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[2]
Jung20_filter['e_f_esc_Lya'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[7]
Jung20_filter['E_f_esc_Lya'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[8]
Jung20_filter['logLLya_int'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[3]
Jung20_filter['e_logLLya_int'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[9]
Jung20_filter['E_logLLya_int'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[10]
#Jung20_filter['logQ_ion'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[4]
Jung20_filter['logN_ion'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[6]
Jung20_filter['e_logN_ion'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[11]
Jung20_filter['E_logN_ion'] = R_s(Jung20_filter['EW_Lya'],Jung20_filter['E_EW_Lya'],Jung20_filter['e_EW_Lya'],Jung20_filter['logLLya'],Jung20_filter['e_logLLya'],Jung20_filter['z_Lya'])[12]
Jung20_filter

ID,logFLya,e_logFLya,S/N,EW_Lya,E_EW_Lya,e_EW_Lya,z_Lya,e_z_Lya,logLLya,e_logLLya,HII_Radii,M_UV,R_s,e_R_s,E_R_s,R_s_com,e_R_s_com,E_R_s_com,f_esc_Lya,e_f_esc_Lya,E_f_esc_Lya,logLLya_int,e_logLLya_int,E_logLLya_int,logN_ion,e_logN_ion,E_logN_ion
,dex(erg / s),dex(erg / s),,Angstrom,Angstrom,Angstrom,,,dex(erg / s),dex(erg / s),pMpc,mag,pMpc,pMpc,pMpc,cMpc,cMpc,cMpc,,,,dex(erg / s),dex(erg / s),dex(erg / s),dex(1 / s),dex(1 / s),dex(1 / s)
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z7_GND_44088,-16.896,0.197,5.2,87.6,23.8,21.2,7.1335,0.0028,42.898,0.203,0.88,-19.9,0.779,0.289,0.289,6.335,2.347,2.347,0.42,0.102,0.114,43.274,0.4827815829528159,0.4827815829528159,53.597,0.483,0.483
z8_GND_22233,-16.866,0.14,7.1,54.5,15.0,12.1,7.3444,0.002,42.959,0.16,0.92,-20.7,0.911,0.428,0.428,7.601,3.568,3.568,0.262,0.058,0.072,43.541,0.6116207951070336,0.6116207951070336,53.864,0.612,0.612
z7_GND_18626,-17.585,0.231,4.6,26.8,14.9,9.8,7.4249,0.0013,42.255,0.222,0.52,-19.8,0.66,0.874,0.874,5.562,7.367,7.367,0.129,0.047,0.072,43.146,1.7257462686567162,1.7257462686567162,53.468,1.726,1.726
z7_GND_42912,-16.836,0.089,10.8,33.2,4.3,4.0,7.5056,0.0007,43.009,0.088,0.96,-21.6,1.077,0.456,0.456,9.16,3.882,3.882,0.159,0.019,0.021,43.807,0.5522088353413656,0.5522088353413656,54.129,0.552,0.552
z7_GND_6330,-17.387,0.171,6.1,15.9,4.4,3.7,7.546,0.0006,42.462,0.172,0.62,-20.7,0.897,1.551,1.551,7.662,13.253,13.253,0.076,0.018,0.021,43.579,2.2536687631027252,2.2536687631027252,53.902,2.254,2.254
z7_GND_16863,-16.724,0.095,10.8,61.3,14.4,11.4,7.5989,0.0011,43.134,0.096,1.07,-21.2,0.947,0.237,0.237,8.139,2.038,2.038,0.294,0.055,0.069,43.665,0.32626427406199027,0.32626427406199027,53.988,0.326,0.326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z7_GND_10402,-17.495,0.25,4.0,6.7,2.7,2.2,7.9395,0.0023,42.415,0.231,0.59,-21.7,1.06,5.843,5.843,9.474,52.23,52.23,0.032,0.011,0.013,43.908,7.182835820895523,7.182835820895523,54.23,7.183,7.183


Los radios de Strömghen calculados a partir de Sobral (Jung20_filter[R_s]) son parecidos a las estimaciones realizadas a partir de la relación de Yajima (Jung20_filter[HII_Radii]). No obstante, debemos de estimar los errores antes para corroborar esta relación. 

Nótese que la notación de R_s y R_s_com se refieren a los radios de Strömgren calculados a partir de al relación de Sobral mientras que el denominado como HII_Radii es el radio de Strömgren calculado a partir de la relación de Yajima.

### 4.1. Errors in Strömgren Radius

Es cierto que hemos calculado los errores del radio de Strömgren para los datos pero no obstante, vamos a hacerlo usando el apquete de `uncertainties` para corroborar que nos sale lo mismo. 

In [19]:
# Recombination coefficinet for T_e = 10⁴K
alphaB = 8.8 * 10**(-87)

# C_H(z) parameter
def C_H(redshift):
    z_values = [6 ,7 , 8]
    C_H_values = [2.37, 2.28, 2.19]
    C_H = np.interp(redshift, z_values,C_H_values) 
    return C_H

# Hydrogen density
def n_H (redshift):
    n_H = 5.878 * 10**66 * (1+redshift)**3
    return n_H

# Escape fraction
f_esc_LyC = 0.2

# Errors in Strömgen Radius
N_unc_e = unumpy.uarray(10**Jung20_filter['logN_ion'].value, (10**Jung20_filter['logN_ion'].value*Jung20_filter['e_logN_ion'].value) / (1 - f_esc_LyC) * f_esc_LyC)
N_unc_E = unumpy.uarray(10**Jung20_filter['logN_ion'].value, (10**Jung20_filter['logN_ion'].value*Jung20_filter['E_logN_ion'].value) / (1 - f_esc_LyC) * f_esc_LyC)
C_H_unc = unumpy.uarray(C_H(Jung20_filter['z_Lya']),0)
n_H_unc = unumpy.uarray(n_H(Jung20_filter['z_Lya']),0)

e_R_s_values =  (3 * N_unc_e / (4 * np.pi * C_H_unc * n_H_unc**2 * alphaB))**(1/3)
E_R_s_values =  (3 * N_unc_E / (4 * np.pi * C_H_unc * n_H_unc**2 * alphaB))**(1/3)

e_R_s = []
E_R_s = []
for i in range(len(e_R_s_values)):
    e_R_s.append(e_R_s_values[i].std_dev)
    E_R_s.append(E_R_s_values[i].std_dev)

Jung20_filter['e_R_s'] = np.round(e_R_s,3) * pMpc
Jung20_filter['E_R_s'] = np.round(E_R_s,3) * pMpc
Jung20_filter['e_R_s_com'] = np.round(e_R_s*(1+Jung20_filter['z_Lya']),3) * cMpc
Jung20_filter['E_R_s_com'] = np.round(E_R_s*(1+Jung20_filter['z_Lya']),3) * cMpc
Jung20_filter[:7]

ID,logFLya,e_logFLya,S/N,EW_Lya,E_EW_Lya,e_EW_Lya,z_Lya,e_z_Lya,logLLya,e_logLLya,HII_Radii,M_UV,R_s,e_R_s,E_R_s,R_s_com,e_R_s_com,E_R_s_com,f_esc_Lya,e_f_esc_Lya,E_f_esc_Lya,logLLya_int,e_logLLya_int,E_logLLya_int,logN_ion,e_logN_ion,E_logN_ion
,dex(erg / s),dex(erg / s),,Angstrom,Angstrom,Angstrom,,,dex(erg / s),dex(erg / s),pMpc,mag,pMpc,pMpc,pMpc,cMpc,cMpc,cMpc,,,,dex(erg / s),dex(erg / s),dex(erg / s),dex(1 / s),dex(1 / s),dex(1 / s)
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z7_GND_44088,-16.896,0.197,5.2,87.6,23.8,21.2,7.1335,0.0028,42.898,0.203,0.88,-19.9,0.779,0.031,0.031,6.335,0.255,0.255,0.42,0.102,0.114,43.274,0.4827815829528159,0.4827815829528159,53.597,0.483,0.483
z8_GND_22233,-16.866,0.14,7.1,54.5,15.0,12.1,7.3444,0.002,42.959,0.16,0.92,-20.7,0.911,0.046,0.046,7.601,0.388,0.388,0.262,0.058,0.072,43.541,0.6116207951070336,0.6116207951070336,53.864,0.612,0.612
z7_GND_18626,-17.585,0.231,4.6,26.8,14.9,9.8,7.4249,0.0013,42.255,0.222,0.52,-19.8,0.66,0.095,0.095,5.562,0.8,0.8,0.129,0.047,0.072,43.146,1.7257462686567162,1.7257462686567162,53.468,1.726,1.726
z7_GND_42912,-16.836,0.089,10.8,33.2,4.3,4.0,7.5056,0.0007,43.009,0.088,0.96,-21.6,1.077,0.05,0.05,9.16,0.421,0.421,0.159,0.019,0.021,43.807,0.5522088353413656,0.5522088353413656,54.129,0.552,0.552
z7_GND_6330,-17.387,0.171,6.1,15.9,4.4,3.7,7.546,0.0006,42.462,0.172,0.62,-20.7,0.897,0.168,0.168,7.662,1.439,1.439,0.076,0.018,0.021,43.579,2.2536687631027252,2.2536687631027252,53.902,2.254,2.254
z7_GND_16863,-16.724,0.095,10.8,61.3,14.4,11.4,7.5989,0.0011,43.134,0.096,1.07,-21.2,0.947,0.026,0.026,8.139,0.221,0.221,0.294,0.055,0.069,43.665,0.32626427406199027,0.32626427406199027,53.988,0.326,0.326
z7_GND_34204,-16.346,0.126,7.9,279.7,80.4,62.5,7.6082,0.003,43.513,0.126,1.44,-20.4,0.84,0.009,0.009,7.235,0.076,0.076,1.0,0.3,0.386,43.513,0.126,0.126,53.835,0.126,0.126


## Appendix A: Saving data

Si se ejecuta el siguiente código, guardaremos y sobreescribiremo los datos. Nótese que lo guardamos en formato ecsv (formato profesional para astrofísica)

In [7]:
ascii.write(Jung20_filter, 'Jung20_filter_v1.dat', format='ecsv', overwrite=True)  

In [18]:
# To LaTeX
data_table = ascii.read('Jung20_filter_v1.dat', format='ecsv')
df = data_table.to_pandas()[:7]
path = '/home/juanan/Escritorio/Juanan/Estudios/Master_en_Astrofisica/Practicas_ESAC/MyJob/latex_tables/Jung20.tex'
cols = ['ID', 'z_Lya', 'EW_Lya', 'logLLya', 'f_esc_Lya', 'R_s', 'HII_Radii']
df.to_latex(buf=path ,columns= cols, float_format='%.2f',header=True, index=False, position= 'H', label = 'Jung20', caption = 'Jung20')